# Checkbox Cropping Pipeline

This notebook implements the checkbox cropping pipeline used in this project.
The pipeline extracts checkbox regions from document images using COCO bounding
box annotations.

The same cropping logic was used to generate:
- `cropped_checkboxes_binary` (full cropped dataset)
- `cropped_checkboxes_binary_small` (reduced and balanced subset)

The reduced dataset was created by applying an additional sampling constraint
after cropping. No changes were made to the cropping logic itself.


In [ ]:
import os
import json
import cv2
import shutil
import random
from tqdm import tqdm
from collections import Counter

# =========================
# CONFIG (BINARY ONLY)
# =========================
BASE_DIR = "../data/raw/Checkbox.v7i.coco"

OUTPUT_DIR_FULL  = "../data/processed/cropped_checkboxes_binary"
OUTPUT_DIR_SMALL = "../data/processed/cropped_checkboxes_binary_small"

SPLITS = ["train", "valid", "test"]
CLASSES = ["checked", "unchecked"]

TARGET_SIZE = 224
PADDING_RATIO = 0.25

# ---- reduce sizes (balanced) ----
LIMITS = {
    "train": 500,
    "valid": 150,
    "test":  200
}

SEED = 42
random.seed(SEED)

# =========================
# RESET OUTPUT DIR (SAFE OVERWRITE)
# =========================
OVERWRITE = True  # set to False to prevent deletion
def reset_output_dir(root_dir):
    if os.path.exists(root_dir):
        if not OVERWRITE:
            raise RuntimeError(
                f"{root_dir} already exists. Set OVERWRITE=True to recreate it."
            )
        shutil.rmtree(root_dir)
    for split in SPLITS:
        for cls in CLASSES:
            os.makedirs(os.path.join(root_dir, split, cls), exist_ok=True)


# =========================
# STEP 1: CREATE FULL CROPPED DATASET
# =========================
print("Resetting FULL output folder (safe overwrite)...")
reset_output_dir(OUTPUT_DIR_FULL)

for split in SPLITS:
    print(f"\nProcessing split: {split}")

    image_dir = os.path.join(BASE_DIR, split)
    ann_file = os.path.join(image_dir, "_annotations.coco.json")

    with open(ann_file, "r") as f:
        coco = json.load(f)

    image_id_to_name = {img["id"]: img["file_name"] for img in coco["images"]}
    category_id_to_name = {cat["id"]: cat["name"] for cat in coco["categories"]}

    counter = 0
    class_counter = Counter()

    for ann in tqdm(coco["annotations"], desc=f"Cropping {split}"):
        class_name = category_id_to_name[ann["category_id"]]

        if class_name not in CLASSES:
            continue

        img_id = ann["image_id"]
        x, y, w, h = map(int, ann["bbox"])

        # Safety clamps
        x = max(0, x)
        y = max(0, y)
        w = max(1, w)
        h = max(1, h)

        img_path = os.path.join(image_dir, image_id_to_name[img_id])

        if os.path.basename(img_path).startswith("."):
            continue

        image = cv2.imread(img_path)
        if image is None:
            continue

        H, W = image.shape[:2]

        # Padding around bbox
        pad_x = int(w * PADDING_RATIO)
        pad_y = int(h * PADDING_RATIO)

        x1 = max(0, x - pad_x)
        y1 = max(0, y - pad_y)
        x2 = min(W, x + w + pad_x)
        y2 = min(H, y + h + pad_y)

        crop = image[y1:y2, x1:x2]
        if crop.size == 0:
            continue

        crop = cv2.resize(crop, (TARGET_SIZE, TARGET_SIZE), interpolation=cv2.INTER_AREA)

        out_name = f"{split}_{img_id}_{counter}.png"
        out_path = os.path.join(OUTPUT_DIR_FULL, split, class_name, out_name)
        cv2.imwrite(out_path, crop)

        counter += 1
        class_counter[class_name] += 1

    print(f"Saved {counter} crops for {split}")
    print("Class distribution:", dict(class_counter))

print("\n✅ FULL binary dataset created successfully.")
print(f"📁 Output saved at: {OUTPUT_DIR_FULL}")

# =========================
# STEP 2: CREATE SMALL BALANCED DATASET
# =========================
print("\n🧹 Resetting SMALL output folder (safe overwrite)...")
reset_output_dir(OUTPUT_DIR_SMALL)

def list_images(folder):
    return [
        os.path.join(folder, f)
        for f in os.listdir(folder)
        if (not f.startswith(".")) and f.lower().endswith((".png", ".jpg", ".jpeg"))
    ]

print("\nCreating reduced dataset (balanced)...")

# Reduction step:
# Limits the number of cropped samples per class per split
# Used only for constructing the reduced dataset

for split in SPLITS:
    per_class_limit = LIMITS[split]
    split_counter = Counter()

    for cls in CLASSES:
        src_folder = os.path.join(OUTPUT_DIR_FULL, split, cls)
        dst_folder = os.path.join(OUTPUT_DIR_SMALL, split, cls)

        files = list_images(src_folder)
        random.shuffle(files)

        keep_n = min(per_class_limit, len(files))
        selected = files[:keep_n]

        for path in selected:
            shutil.copy2(path, os.path.join(dst_folder, os.path.basename(path)))

        split_counter[cls] = keep_n

    print(f"\n{split.upper()} reduced distribution:", dict(split_counter))

print("\n✅ SMALL binary dataset created successfully.")
print(f"📁 Output saved at: {OUTPUT_DIR_SMALL}")
print("Final limits used:", LIMITS)


🧹 Resetting FULL output folder (safe overwrite)...

Processing split: train


Cropping train: 100%|██████████| 4403/4403 [00:19<00:00, 228.54it/s]


Saved 4403 crops for train
Class distribution: {'unchecked': 2374, 'checked': 2029}

Processing split: valid


Cropping valid: 100%|██████████| 978/978 [00:04<00:00, 240.13it/s]


Saved 978 crops for valid
Class distribution: {'unchecked': 451, 'checked': 527}

Processing split: test


Cropping test: 100%|██████████| 714/714 [00:03<00:00, 236.02it/s]


Saved 714 crops for test
Class distribution: {'checked': 233, 'unchecked': 481}

✅ FULL binary dataset created successfully.
📁 Output saved at: ../processed/cropped_checkboxes_binary

🧹 Resetting SMALL output folder (safe overwrite)...

📌 Creating reduced dataset (balanced)...

TRAIN reduced distribution: {'checked': 500, 'unchecked': 500}

VALID reduced distribution: {'checked': 150, 'unchecked': 150}

TEST reduced distribution: {'checked': 200, 'unchecked': 200}

✅ SMALL binary dataset created successfully.
📁 Output saved at: ../processed/cropped_checkboxes_binary_small
Final limits used: {'train': 500, 'valid': 150, 'test': 200}
